# Structure Manipulation

In this tutorial we will learn how to manipualte chemical structures using [ASE](https://wiki.fysik.dtu.dk/ase/ase/atoms.html#module-ase.atoms). ASE is one of the most important tools to be comfortable with to be succesful in the Musgrave group. It allows us to quickly manipulate structures (cutting surfaces, adding adsorbates, rotating molecules... etc.) and run DFT calculations using JDFTx. Furthermore, ASE is the package that underpins the high throughput script stack in our group, so it is necessary to understand to be able to troubleshoot high throughput problems. 
This notebook will be executed with python, so no terminal is necessary.

In [3]:
from ase import Atoms #need to import ASE functions 
from ase.visualize import view

### Atoms

ASE uses an object called ```Atoms``` to represent all chemical structures. Just like any other object, we can instantiate one by calling the name of the object. Let's say we want to create an N2 molecule using ASE. To do so, we can instantiate the object using the names of the elements in the objects followed by their positions

In [4]:
N2_molecule = Atoms('NN', positions=[(0,0,0),(0,0,1.1081)])

It may not seem like we've done anything because we can't tell if any structure was actually made. To visualize the sctructure, we can use the ```view``` function that we imported above.

In [5]:
view(N2_molecule)

<Popen: returncode: None args: ['C:\\Users\\coopy\\anaconda3\\envs\\spy-env\...>

We can see now that the ```N2_molecule``` instance of the ```Atoms``` structure does in fact represent our N2 molecule. However, it's quite inconvenient to create an Atoms object by specifying all the elements and positions manually. Instead, let's read in a POSCAR structure and convert it into an Atoms object.

### Reading and Writing

In [6]:
from ase.io import read, write

We are going to read in our N2 molecule directly from the github ```files/N2/``` directory in the tutorial files. To read in the POSCAR, all we need to do is pass the path to the ```read()``` function

In [7]:
N2_molecule = read("../Files/N2/POSCAR")
view(N2_molecule)

<Popen: returncode: None args: ['C:\\Users\\coopy\\anaconda3\\envs\\spy-env\...>

You'll notice that this molecule looks different than the one we visualized initially. In the first visualization, there was no unit cell boundaries and the molecule existed in an infinite vacuum. However, when we initialized our structure from a POSCAR, ASE depicted it as a periodic unit cell containing an N<sub>2</sub> molecule. For all of the POSCARs we are working with, they will always be periodic unit cells.

Now that we've read the file, let's save it as a POSCAR to a more accessible location. In this case, I'm going to save it to my Desktop. To do that, you'll need the path to your Desktop. In my case, it's ```C:\Users\coopy\OneDrive - UCB-O365\Desktop```. We can use the ```write()``` function imported above to save the file. Make sure you specify the file format since ASE can write to a variety of [file formats](https://wiki.fysik.dtu.dk/ase/ase/io/io.html).

In [8]:
path = "C:\\Users\\coopy\\OneDrive - UCB-O365\\Desktop\\POSCAR" # Single backslash won't work
# need to speciy POSCAR at the end of the path string to give it the file name
write(path, N2_molecule, format='vasp')

You should have noticed that a ```POSCAR``` file showed up on your desktop. These ```read()``` and ```write()``` functions are quite general and powerful, and will be used countless times throughout our high throughput script stack.

### Supercells

Now we'll look at repeating the unit cell along its lattice directions to create a supercell. A nitrogen molecule supercell doesn't really make sense, so we're going to start looking at bulk Pt instead. Start by initializing the structure from the ```POSCAR_bulk``` in ```../Files/Pt_111/```

In [9]:
# Try to initialize this structure yourself
# The path you'll need is path="../Files/Pt_111/POSCAR_bulk"
# Make "Pt" the variable name for your structure 

If you view this structure, you'll notice it looks a little odd. First, there are only 4 spheres in our cubic lattice, because the view function isn't showing the repeated atoms on the periodic boundaries. If you want to view it with those atoms to help see it better, I recommend using VESTA. You can use the ```write()``` function to save it to your Desktop and drag it into VESTA

<img src="../Pictures/ASE/Pt_Vesta.png" alt="Pt Vesta">

In VESTA it is more apparent that we do in fact have a cubic crystal structure of Pt. What [bravais lattice](https://byjus.com/chemistry/bravais-lattice/) is this unit cell? Are we using [the primitive lattice vectors](https://eis.hu.edu.jo/ACUploads/10010/crystal_structure_3.pdf) (first page of this link) to specify it? Both of these questions are more easily answered using the ```view()``` function.

Now, let's make it a supercell in all 3 lattice directions. In order to do so, we just need to call the function ```make_supercell()```

In [ ]:
from ase.build import make_supercell
scale_factor = [[2,0,0],[0,2,0],[0,0,2]] # tells ASE to double the unit cell in each (x,y,z) direction
Pt_super = make_supercell(Pt, scale_factor)
view(Pt_super)

We can see that the unit cell got bigger and now looks more crystalline. For some proof that we did double the unit cell in each direction, let's print the lattice vectors and list of ions in the original unit cell and the supercell. The lattice vectors should double in all directions and the number of atoms should be multiplied by 2<sup>3</sup>.

In [ ]:
print(Pt.get_cell()) #get unit cell and atomic symbols for original Pt unit cell
print(Pt.get_chemical_symbols())
print(len(Pt.get_chemical_symbols()))

In [ ]:
print(Pt_super.get_cell()) 
print(Pt_super.get_chemical_symbols())
print(len(Pt_super.get_chemical_symbols()))

Indeed, we successfully doubled the unit cell in all directions. Notice as well that there are a bunch of built-in methods that can [give you data](https://wiki.fysik.dtu.dk/ase/ase/atoms.html) in a readable format about ```Atoms``` objects.

### Pymatgen

Now that we have a bulk unit cell, we can cut it into a surface. While we can do this with ASE, it's much easier to use [Pymatgen](https://pymatgen.org/introduction.html). Pymatgen is a completely different python package than ASE they cannot be mixed together. While ASE uses the ```Atoms``` object to represent chemical structures, pymatgen uses the ```Structure``` object. ASE methods and functions cannot be used on pymatgen structures and vice versa. Fortunately, both packages allow us to convert in between pymatgen and ASE. 

In [ ]:
from pymatgen.io.ase import AseAtomsAdaptor

pymatgen_structure = AseAtomsAdaptor.get_structure(Pt_super)
print(type(pymatgen_structure))

In [1]:
from pymatgen.vis.structure_vtk import StructureVis
visualize = StructureVis().set_structure(pymatgen_structure)

NameError: name 'pymatgen_structure' is not defined

In [ ]:
from pymatgen.core.surface import SlabGenerator